In [1]:
# import pandas to deal with csv dataframe
import pandas as pd
# import urllib to make connection to websites and get source code
from urllib.request import Request, urlopen
# import beautiful soup to extract information from source code
from bs4 import BeautifulSoup as bs
# import regular expression
import re

In [2]:
# read the latest csv file
birds_df = pd.read_csv("birds_with_all_colour_added_Apri_5_2017.csv")

In [3]:
# check the column names
birds_df.columns

Index(['Idx', 'Common name', 'Binomial', 'Category', 'Order', 'Family',
       'colour1', 'colour2', 'colour3', 'min_length', 'max_length',
       'min_weight', 'max_weight', 'Voice', 'Description', 'Diet', 'Sources'],
      dtype='object')

In [4]:
# url of list of birds of each state
url_list = ["http://avibase.bsc-eoc.org/checklist.jsp?lang=EN&p2=1&list=avibase&synlang=&region=AUvi&version=text&lifelist=&highlight=0",
           "http://avibase.bsc-eoc.org/checklist.jsp?lang=EN&p2=1&list=avibase&synlang=&region=AUns&version=text&lifelist=&highlight=0",
           "http://avibase.bsc-eoc.org/checklist.jsp?lang=EN&p2=1&list=avibase&synlang=&region=AUql&version=text&lifelist=&highlight=0",
           "http://avibase.bsc-eoc.org/checklist.jsp?lang=EN&p2=1&list=avibase&synlang=&region=AUca&version=text&lifelist=&highlight=0",
           "http://avibase.bsc-eoc.org/checklist.jsp?lang=EN&p2=1&list=avibase&synlang=&region=AUno&version=text&lifelist=&highlight=0",
           "http://avibase.bsc-eoc.org/checklist.jsp?lang=EN&p2=1&list=avibase&synlang=&region=AUsa&version=text&lifelist=&highlight=0",
           "http://avibase.bsc-eoc.org/checklist.jsp?lang=EN&p2=1&list=avibase&synlang=&region=AUta&version=text&lifelist=&highlight=0",
           "http://avibase.bsc-eoc.org/checklist.jsp?lang=EN&p2=1&list=avibase&synlang=&region=AUwa&version=text&lifelist=&highlight=0"
           ]

In [5]:
# create a string list of 8 states
location_str = ['vic', 'nsw', 'qld', 'act', 'nt', 'sa', 'tas', 'wa']

In [6]:
# initialize 8 DataFrame for each state
location_df = [pd.DataFrame() for dummy in range(8)]

In [7]:
# make 8 dataframe with columns ['common_name', 'binomial', 'status'] getting from url
for idx in range(8):
    url = url_list[idx]
    location_df[idx] = pd.read_html(url)
    location_df[idx] = location_df[idx][1].copy()
    # only keep rows from 15 to the second last.
    location_df[idx] = location_df[idx][15:-1]
    # remove all other columns
    location_df[idx] = location_df[idx][[0,1,3]]
    location_df[idx].columns = ['common_name', 'binomial', 'status']
    location_df[idx] = location_df[idx][location_df[idx]['binomial'].notnull()]

In [8]:
# initialize state columns in original dataframe
for state in location_str:
    birds_df[state] = pd.np.nan

In [9]:
# check if a bird exist in one state, if it exists change the value to 1.0, otherwise leave it null
for state_idx in range(len(location_str)):
    state_df = location_df[state_idx]
    state_str = location_str[state_idx]
    for idx in birds_df.index:
        if birds_df.ix[idx, 'Binomial'] in state_df.binomial.values:
            birds_df.ix[idx, state_str] = 1

In [10]:
# check the combined dataframe
birds_df.head()

,Idx,Common name,Binomial,Category,Order,Family,colour1,colour2,colour3,min_length,...,Diet,Sources,vic,nsw,qld,act,nt,sa,tas,wa
0,0,Southern cassowary,Casuarius casuarius,Cassowaries,Casuariformes,Casuariidae,black,blue,red,NaN,...,Southern Cassowary feeds mainly on fallen frui...,HANDBOOK OF THE BIRDS OF THE WORLD vol 1 by J...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,1,Emu,Dromaius novaehollandiae,Emus,Casuariformes,Dromaiidae,brown,NaN,NaN,150.0,...,"The Emu is omnivorous, but it feeds regularly...",HANDBOOK OF THE BIRDS OF THE WORLD vol 1 by Jo...,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0
2,6,Orange-footed scrubfowl,Megapodius reinwardt,Mound-builders,Galliformes,Megapodidae,brown,blue,NaN,35.0,...,"Orange-footed Scrubfowl feeds mainly on seeds,...",HANDBOOK OF THE BIRDS OF THE WORLD Vol 2 by J...,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0
3,10,Indian peafowl,Pavo cristatus,Pheasants,Galliformes,Phasianidae,Blue,green,NaN,180.0,...,"The Indian Peafowl is omnivorous, feeding on p...",HANDBOOK OF THE BIRDS OF THE WORLD Vol 2 by Jo...,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0
4,13,Common pheasant,Phasianus colchicus,Pheasants,Galliformes,Phasianidae,brown,tan,NaN,75.0,...,"Common Pheasant feeds on cereal grains, seeds,...",HANDBOOK OF THE BIRDS OF THE WORLD Vol 2 by J...,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0


In [11]:
# save the combined the dataframe into a new csv file
birds_df.to_csv("birds_April_6.csv")